## 一.  项目背景
PaddleDetection团队针对PP-YOLOE+提供了少样本学习的方案，引入了Compare-Contrast算法。在预训练模型的基础上，仅通过极少的标注数据即可训练得到较好的模型效果。

本项目基于PaddleDetection v2.6中的PPYOLOE+网络，通过10个标注样本（10shots）的训练，实现4类路标（crosswalk，speedlimit，stop，trafficlight）检测，目的是体会PPYOLOE+小样本目标检测在实际场景中的应用。
### 1.1 检测结果
![](https://ai-studio-static-online.cdn.bcebos.com/232bcd4e42e74ce0b270ff4efebb34d0232e6c7d40ce45ab99303ddb53c290f4)
![](https://ai-studio-static-online.cdn.bcebos.com/cae9a41b75ed4be8a5a799863e373bedd6887e80f5894f4aadf6919d5d0733c2)
![](https://ai-studio-static-online.cdn.bcebos.com/6018c3f1d3624d028e0a35057b69d3afb9abae7948d543279b02ad11037acae5)



### 1.2 小样本目标检测
目前基于深度学习的目标检测模型需要大量的标注样本进行训练，但在实际场景当中，往往很难获取到相应规模的高质量标注样本。小样本目标检测是传统目标检测技术与小样本学习技术的融合，通过少量的标注样本学习具有较好泛化性能的检测模型，降低对大规模标签数据的依赖。


## 二. Contrast算法介绍

Contrast算法来自于Google的一篇论文[Supervised Contrastive Learning](https://arxiv.org/abs/2004.11362)

监督对比学习（Supervised Contrastive Learning）是基于自监督对比学习（Self Supervised Contrastive Learning）进行的改进，总结来讲就是设计了一个**supervised contrastive loss**让同类feature接近，异类feature远离，提高提取的feature质量，在feature这个维度上提升分类网络的性能。
### 2.1 Self Supervised Contrastive Learning
这种方法的本质是：分别用与图片来源相同的图片的feature、与图片来源不同的图片的feature，跟图片的feature进行对比，然后让来源相同的图片feature越接近，来源不同的图片feature越远。这种方式虽然能学到不错的feature，但有一个不足是：没有考虑到属于同一个类的不同图片之间的feature的相关性。例如下图所示的情况：左边的小狗和下面的小狗属于同一类但是距离比较远，这显然不合理。
![](https://ai-studio-static-online.cdn.bcebos.com/c36d2b915f1a49799a461426fd64260af588c1d2622a4561b9231e2c67c994a9)
### 2.2 Supervised Contrastive Learning
基于以上问题，为了让同一类的图片feature距离接近，Google设计了一个**supervised contrastive loss**，将对比学习和监督学习相结合，达到更好的效果。下图所示同一小狗类相互接近，猫类和小狗类相互分开。
![](https://ai-studio-static-online.cdn.bcebos.com/6d81a04facad44a1976b7f66b9351338447eebdbd1184fb498bdf21deedfd8ef)
### 2.3 Supervised Contrastive loss
该loss表达的含义是：

1.对于任意图片i与图片属于同类的所有其它图片的feature，与图片i的feature的余弦距离的总和，越大越好

2.与图片i不属于同类的所有其它图片的feature，与图片i的feature的余弦距离的总和越小，越小越好
![](https://ai-studio-static-online.cdn.bcebos.com/47900f5ece6e4d8e9640b2dcf0986409976ed1fd963a410f9e1fd53b786955bd)


### 2.4 PP-YOLOE+s-Contrast
PP-YOLOE+s在Head损失权重上加入Supervised Contrastive loss，通过引入Contrast算法，Box AP得到了2.4的提升
| 骨架网络             | 网络类型       | 每张GPU图片个数 | 每类样本个数 | Contrast |  Box AP  |
| :------------------- | :------------- | :-----: | :-----: | :------------: | :-----: |
| PPYOLOE_crn_s             | PPYOLOE         |    1    |     30    |  False  |  15.4  |
| PPYOLOE_crn_s             | PPYOLOE         |    1    |     30     |  True  |  17.8 |

![](https://ai-studio-static-online.cdn.bcebos.com/b445efe3458c42eba40dc781748d650b1534398fe5964ef2ad3c6c08774e4f65)

## 三. 数据集介绍

Kaggle上的[road-sign-detection](https://www.kaggle.com/datasets/andrewmvd/road-sign-detection)比赛数据包含877张图像，数据类别4类：crosswalk，speedlimit，stop，trafficlight，用于道路标志检测。本项目分别从原始数据集中每类选取10shots即每类都有10个训练样本训练。**这里的10shots并不是指10张图片，而是10个标注框样本**，左图为训练样本，categories表示类别数，annotations表示标注框个数即训练样本数，如下右图训练时一张图上有多个标注样本

![](https://ai-studio-static-online.cdn.bcebos.com/45ec2d36ef7c422c8fef52b17af0258f6d9af9e34dd846c2bb2c936fc29dacf8)
![](https://ai-studio-static-online.cdn.bcebos.com/2e5cb1893d2c4dd78cee61cd9b9080fd6c81be8c1f1a432b880f9ba87260ab71)


In [ ]:
# 解压数据集（仅运行一次）
!tar -xvf data/data206532/roadsign_coco.tar.gz -C ./data/

In [2]:
# 组织数据目录
!mkdir MyDataset
!mkdir MyDataset/JPEGImages

In [3]:
!mv data/roadsign_coco/images/* MyDataset/JPEGImages/
!mv data/roadsign_coco/annotations/* MyDataset/

## 三、安装PaddleDetection以及依赖

In [4]:
!git clone https://gitee.com/paddlepaddle/PaddleDetection.git

正克隆到 'PaddleDetection'...
remote: Enumerating objects: 257119, done.
remote: Counting objects: 100% (2116/2116), done.
remote: Compressing objects: 100% (1231/1231), done.
remote: Total 257119 (delta 1302), reused 1595 (delta 872), pack-reused 255003
接收对象中: 100% (257119/257119), 413.58 MiB | 17.77 MiB/s, 完成.
处理 delta 中: 100% (210330/210330), 完成.
检查连接... 完成。


In [1]:
%cd PaddleDetection

/home/aistudio/PaddleDetection


In [ ]:
# 安装依赖
!pip install -r requirements.txt --user
!python setup.py install --user

## 四、模型选型
PaddleDetection提供了多种模型进行选择，由于是few-shot（小样本）目标检测，因此我们找到configs文件下的few-shot文件夹，基于体验ppyoloe对小样本的检测效果，本次采用**ppyoloe_plus_crn_s_80e_contrast_pcb.yml**模型文件训练，包含以下几个配置文件，为了方便更改，将其复制到aistudio目录下。

![](https://ai-studio-static-online.cdn.bcebos.com/f853cbd0a4484828a7f361898b88ca415c8f3b71b89741b4af86de15c7fa7eda)


In [3]:
# 训练配置文件覆盖
!cp ../ppyoloe_plus_crn_s_80e_contrast_pcb.yml configs/few-shot/ppyoloe_plus_crn_s_80e_contrast_pcb.yml

### 4.1 主要更改部分
由于训练文件**ppyoloe_plus_crn_s_80e_contrast_pcb.yml**下半部分已经将配置文件重写覆盖，因此只需要修改数据集路径和部分参数即可，这里将snapshot_epoch:改为5，即每5轮对模型验证保存一次，将迭代轮数改为60轮。

![](https://ai-studio-static-online.cdn.bcebos.com/6b9648c4de6947688df20c682855bb38040f0d7a4cbd432b8dcce9a6f77096ad)



![](https://ai-studio-static-online.cdn.bcebos.com/d6320aa291de46abaf05c48f55960ee9a1dbe6d7633a4e73bd2d6d64df10f725)


### 4.2 开始训练
* 训练45轮左右模型达到最优map为0.635

* V100训练60轮时间大概只需要2分钟

* 继续训练模型map下降，模型不在收敛，这里应该是过拟合了，毕竟只有10个样本作为数据支撑

![](https://ai-studio-static-online.cdn.bcebos.com/d8614f0823834af097230d55b0e49a0744d238df75d24ba18578a386e66a62e8)



In [ ]:
# -c 参数表示指定使用哪个配置文件
# --eval 参数表示边训练边评估，训练过程中会保存验证效果最佳的checkpoint
!python tools/train.py -c configs/few-shot/ppyoloe_plus_crn_s_80e_contrast_pcb.yml --amp --eval --use_vdl=True --vdl_log_dir=./visdrone/

### 4.3 训练可视化
![](https://ai-studio-static-online.cdn.bcebos.com/089f128cf19a4c34a21c54c0b121b9a0fe0496a02ce44b7d882342c62a31a5fb)
![](https://ai-studio-static-online.cdn.bcebos.com/35ee8c316b854c8ba1eec41f25a445106d895facbbf14fd1a325ad2d42ea16b6)


### 4.3 模型评估

In [ ]:
# -c 参数表示指定使用哪个配置文件
# -o 参数表示指定配置文件中的全局变量（覆盖配置文件中的设置）
!python tools/eval.py -c configs/few-shot/ppyoloe_plus_crn_s_80e_contrast_pcb.yml -o weights=output/ppyoloe_plus_crn_s_80e_contrast_pcb/best_model.pdparams

## 五、推理一张图片

![](https://ai-studio-static-online.cdn.bcebos.com/302043305fb54bc9b8aa7970b8cd34c09cfb92c0b065415e84e9b511a95bdce3)


In [ ]:
# -c 参数表示指定使用哪个配置文件
# --infer_img 参数指定预测图像路径
# 推理后图片保存在output/road69.png
!python tools/infer.py \
-c configs/few-shot/ppyoloe_plus_crn_s_80e_contrast_pcb.yml \
-o weights=output/ppyoloe_plus_crn_s_80e_contrast_pcb/best_model.pdparams \
--infer_img=../MyDataset/JPEGImages/road69.png

## 六、总结展望

* PaddleDetection引入的Contrast算法在小样本目标检测领域取得了2.4AP提升，值得学习。在训练自己的模型时，也可以试试这个supervised contrastive loss

* 本文利用paddeledet框架通过10个样本训练出路标检测模型，训练45轮左右模型达到最优map为0.635，在小样本目标检测算法上取得较好的结果

* 有的路标检测不准确，或者召回率较低，没有检测出相应目标，模型还有较大进步空间可更改超参数或者数据增强进行多次训练

* 小样本目标检测应用广泛，如何在数据量较少的情况下训练出有效的模型是值得研究的，目前主流的方向为数据扩充，半监督，迁移学习，元学习，注意力机制等

## 七、参考资料
* [有监督对比学习：Supervised Contrastive Learning](https://zhuanlan.zhihu.com/p/136332151)

* [Supervised Contrastive Learning](https://arxiv.org/abs/2004.11362)

* [PP-YOLOE+少样本学习方案](https://github.com/PaddlePaddle/PaddleDetection/tree/release/2.6/configs/few-shot) 
